In [1]:
"""The aim is to extract data from the bankstatement, The statement is divided into 3 sections
the haed, The body and the end.
"""

'The aim is to extract data from the bankstatement, The statement is divided into 3 sections\nthe haed, The body and the end.\n'

In [1]:
import cv2
import numpy as np
import pytesseract as tess
tess.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image

In [26]:
#Converting file to image
from pdf2image import convert_from_path
image = convert_from_path('account_statement.pdf')
for i in range(len(image)):
      # Save pages as images in the pdf
    image[i].save('page'+ str(i) +'.png')

#Get pages from file 
page_number = len(image)

In [2]:
def extract_data(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        threshed = cv2.adaptiveThreshold(gray,255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,17)
        text1 = tess.image_to_data(threshed,output_type='data.frame')
        text2 = tess.image_to_string(threshed)
        text = text1[text1.conf != -1]
        return text2

In [3]:
#Iterate through the pages 

for i in range(page_number) :
    if (i == 0):
        #Dealing with the Head

        #Extracting from the first page
        image0 = cv2.imread('page'+str(i)+'.png')

        #Crop firstpage
        image0= image0[750:2000,40:2000]

        #Extract the data 
        initial_data = extract_data(image0)

        #Cleaning the data 
        #2.Implement the function 
        txt_value = extract_data(image0)
        arrValue = txt_value.split("\n")

        #3.Finding Posting Date
        index_Posting = 0
        for x in arrValue :
            index_Posting +=1
            # find Posting Date
            if x =="Posting Date" :
                break

        #4.Finding Transaction
        index_transaction = 0
        for x in arrValue :
            index_transaction +=1
            # find Posting Date
            if x =="Transaction" :
                break

        #5.Create a list that copies from Posting Date to transaction Date
        Posting_list = arrValue[index_Posting: index_transaction-1]
        Posting_list_copy = Posting_list
        """We are going to a copy because we will need the orginal format of the list 
        to make more manipulation"""
        #getiing rid of empty spaces
        Posting_list_copy = [x for x in Posting_list_copy if x]

        #6.Extracting Transaction Date
        index_description = 0
        for x in arrValue :
            index_description +=1
            # find Posting Date
            if x =="Description" :
                break 

        #7.Finding Money In
        index_MoneyIn = 0
        for x in arrValue :
            index_MoneyIn +=1

            # find Posting Date
            if x =="Money In (R)" :
                break

        #8.Create a list that copies from transactio Date to Description
        transaction_list = arrValue[index_transaction+1:index_description-2]

        #9.Getiing rid of empty spaces
        transaction_list = [x for x in transaction_list if x]
        len(transaction_list)

        #10.Extracting description
        description_list = arrValue[index_description:index_MoneyIn-1]

        #11.Getiing rid of empty spaces
        description_list = [x for x in description_list if x]

        #12.Delete the first character
        Posting_list.pop(0)

        #13.Getting Balance Data
        image0 = cv2.imread('page'+str(i)+'.jpg')
        image0 = image0[790:2000,1400:1800]
        balance_data = extract_data(image0)
        balance_data = balance_data.split("\n")
        balance_list = [x for x in balance_data if x]
        #Deleting the first character
        balance_list.pop(0)
        # #Deleting the last charcter
        balance_list.pop(len(balance_list)-1)
        
        #14.Try to identify the indices spaces below them
        counter = 0
        new_line_position = []

        for x in Posting_list :
            counter +=1
            # find Posting Date
            if x =="" :
                new_line_position.append(counter)
        
        #15.Deleting additional values
        new_list = []
        for x in new_line_position :
            for y, item in enumerate(description_list):
                
                if x == y :
                    new_list.append(description_list[y-1])

        #16.Remove duplicates
        new_list = list(dict.fromkeys(new_list))
        #y is going to the counter in our loop
        for x in new_list :
            counter += 1
            for y, z in enumerate(description_list): 
                if x == z :
                    description_list[y-1] =  description_list[y-1] +" " + description_list[y]
                    description_list[y] = ""

        #17.Getiing rid of empty spaces
        description_list = [x for x in description_list if x]

        
    elif (i== (page_number-1)):
        #Dealing with the end 
        print("Final Page")
    else :
        #Dealing with the Body
        image0 = cv2.imread('page'+str(i)+'.jpg')

        #Get posting date
        cropped_image_posting = image0[40:2050, 60:250]
        posting_data = extract_data(cropped_image_posting)
        posting_arr = posting_data.split("\n")
        #Delete the last character
        posting_arr.pop(len(posting_arr)-1)

        #Get transaction date 
        cropped_image_transaction = image0[40:2050, 250:400]
        transaction_data = extract_data(cropped_image_transaction)
        transaction_arr = transaction_data.split("\n")
        transaction_arr.pop(len(transaction_arr)-1)

        #Get description
        cropped_image_description = image0[40:2050,400:800]
        description_data = extract_data(cropped_image_description)
        description_arr = description_data.split("\n")
        # description_arr = [x for x in description_arr if x]
        description_arr.pop(len(description_arr)-1)
    

        #This can be implemented once we have transaction date and description
        #Identify the indices spaces below them
        counter = 0
        new_line_position = []
        for x in posting_arr :
            counter +=1
            # find Posting Date
            if x =="" :
                new_line_position.append(counter)

        
        # #15.Deleting additional values
        new_list = []
        for x in new_line_position :
            for y, item in enumerate(description_arr):
                if x == y :
                    new_list.append(description_arr[y-1])
        
        #16.Remove duplicates
        new_list = list(dict.fromkeys(new_list))
        
       
    

NameError: name 'page_number' is not defined

In [24]:
new_list

['5813)',
 'Payment Received: 4walls Salary Tre',
 'ATM Cash Withdrawal Fee',
 'ATM Cash Withdrawal Atm3 Pretoria',
 'SMS Notification Fee',
 '',
 'Shoprite Mamelodi (Card 5813)',
 'Osbro Cash and Carry Pretoria (Car¢']